# Analysis of the Bananas

* Quality Score: A numerical score, likely on a scale of 1-4 that rates the overall quality of the banana sample
* Ripeness: How ripeness the banana is.
* Sugar Content Brix: Amount of sugar measured in degrees Brix.
* Firmness (kgf): It's associated with th texture of the banana (hard or soft).
* Tree age: The age of the banana tree.
* Altitude: It's the altitude at which bananas plants are growing, measured in meter above the sea. There are bananas that grow better in a certain altitude and conditions that others.
* Rainfall (mm): Amount of rainfall in the zone where the bananas were harvested. A higher amount of rainfall may indicate favorable growing conditions for bananas.
* Soil nitrogen: Measured in part per million (ppm). It's the concentration of nitrogen in the soil where the bananas were harvested. This variable is a crucial nutrient for plant growth.


In [1]:
# Load the data
bananas <- read.csv("../data/banana.csv")

The goal of this analysis is to identify which variables are important at the time to determine the quality of a banana. And then build a regression model to predict the quality of an unseen record. 

In [2]:
# Filter those variables that are not numeric to build the feature's matrix.
numeric_cols <- c()
for (i in 1:ncol(bananas)){
    if (class(bananas[, i]) == "numeric"){
        numeric_cols <- c(numeric_cols, i)
    }
}
bananas_num <- bananas[, numeric_cols]
# Build a regression model by setting the quality_score as the target and the rest as explained variables. 
model <- lm(quality_score~., data = bananas_num)
model


Call:
lm(formula = quality_score ~ ., data = bananas_num)

Coefficients:
       (Intercept)      ripeness_index  sugar_content_brix        firmness_kgf  
        -1.997e+00           2.002e-01           1.565e-01          -1.565e-02  
         length_cm            weight_g      tree_age_years          altitude_m  
         3.975e-02           3.207e-05           2.286e-04           3.833e-06  
       rainfall_mm   soil_nitrogen_ppm  
         1.692e-06          -6.756e-05  


In this case we consider the rest as predicted variables, but this lead that the regression model is not good to predict the quality score of a banana. Therefore, it will be explained how to know when a model is good or not and how to improve it. 

In [3]:
# Analyse the domain of the target variable
summary(bananas_num$quality_score)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.920   2.090   2.440   2.465   2.850   3.890 

In [4]:
# Another form to calculate the coefficientes of the model by using the canonical equation of regression 
Y <- bananas_num[, "quality_score"] # Dependent or target variable
X <- bananas_num[, -1] # Exclude the target variable
ones_col <- matrix(rep(1, nrow(bananas_num)), nrow = nrow(bananas_num))
# Add a column with ones
X <- as.matrix(cbind(ones_col, X)) # Reserved for intercept (\beta0)
# Get the coefficients vector
betas <- solve(t(X) %*% X) %*% t(X) %*% Y
# Now, we can compute the predictive values of the observations.
y_hat <- X %*% betas
# To understand the content of y_hat
cat("The expected quality score for the banana associated with observation 1:", y_hat[1])

The expected quality score for the banana associated with observation 1: 1.85639

In [5]:
# Calculate the mean of the residuals or errors that our model committed.
# If it's accuracy the value has to be close to 0.
SSR <- sum((Y - y_hat)^2)
# To calculte the mean of the sum of squares it's important to know the number of
# degrees of freedom that we have now (n-10), because we have estimated 10 betas.
n <- nrow(bananas_num) - 10
MSSR <- 1/n * SSR
MSSR

[1] 0.004727049

Despite the fact that we have choosen all the numeric variables to build the model, the predictions are very close to real observations.
$$
MSSR = 4.72 \cdot 10^{-3} 
$$

In [6]:
# To do the same process by using the model object
MSSR <- sum(model$residuals^2) / model$df.residual
Y_hat <- model$fitted.values # Predictive values for each observation

Another important result that it can be computed by using the expression that we use to calculate the coefficients vector:
$$
Y^{\hat{}} = X\beta^{\hat{}} = X(X^tX)^{-1}X^tY = HY;\quad H =  X(X^tX)^{-1}X^t
$$
The famuous Hat matrix. This stores the leverages ($h$) of each observation along its diagonal. These leverage values measure how far an observation is from the "center" of the predictor variable space, considering the multivariate relationships between variables.

A high leverage value indicates that the observation has a greater potential to influence the estimation of the regression coefficients. This might lead to a hyperplane that poorly represents the overall data structure.

In [13]:
H <- X %*% solve(t(X) %*% X) %*% t(X)
leverages <- diag(H)

A leverage $h_{ii}$ is considered influential if it's greater than twice average leverage.
$$
h_{ii} \text{ is influential } \iff h_{ii} > 2(\frac{1}{n}\Sigma_{i=1}^n h_{ii})
$$
And there is a property that sets that the total sum of the leverages is equal to the number of coefficientes that we've estimated, that means $\Sigma_{i=1}^n h_{ii} = k + 1$.

$$
h_{ii} \text{ is influential } \iff h_{ii} > 2(\frac{\rho}{n});\quad \rho = k + 1
$$


In [31]:
p <- model$rank # Number of estimated coefficientes for the model 
n <- nrow(bananas_num) # Numbe of observations
# To search for the influential points
influential_points <- which(leverages > (2 * p/n))
if (sum(influential_points) == 0){
    print("There are not influential points")
}else{
    cat("Influential points", influential_points)
}

[1] "There are not influential points"


In [53]:
# Print the summary of the coefficients of the model
summary(model)


Call:
lm(formula = quality_score ~ ., data = bananas_num)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.217729 -0.055512  0.001806  0.052638  0.146401 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)        -1.997e+00  2.610e-02 -76.504   <2e-16 ***
ripeness_index      2.002e-01  1.248e-03 160.408   <2e-16 ***
sugar_content_brix  1.565e-01  1.072e-03 146.038   <2e-16 ***
firmness_kgf       -1.565e-02  1.692e-03  -9.252   <2e-16 ***
length_cm           3.975e-02  3.806e-04 104.441   <2e-16 ***
weight_g            3.207e-05  4.440e-05   0.722   0.4704    
tree_age_years      2.286e-04  4.181e-04   0.547   0.5846    
altitude_m          3.833e-06  5.113e-06   0.750   0.4536    
rainfall_mm         1.692e-06  3.864e-06   0.438   0.6614    
soil_nitrogen_ppm  -6.756e-05  4.068e-05  -1.661   0.0971 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.06875 on 990 degrees of freedom
Mul

Understanding this table is really important to know, because you can see which independent variables are necessary to use in the model, the mean error that it commits at the time to get the predictive values and the predictive capacity of the model. So I will explain it in detail:

* Residuals's section: How the errors or residuals that the model committed to compute the predictive values vary. As we can see there is a banana whose estimated quality rate is greater than the real by 0.21 units. Remind that an residual is $e_i = y_i - y_i^{\text{hat}}$. It's important that the residuals are close to 0, sinde it's a indicator that the estimated values are close of real ones.
* Coefficients's section:
  * Esimate: Estimated value of the coefficient of an explanatory variable. It represents the influence that a variable has at the time to explain the variability of the response or dependent variable.
  * Std. Error: The error committed when calculating the estimator
    
  To understand the two last columns it's important to know what hypothesis contrast is being used here:
  $$
  \begin{cases}H_0:\beta_i = 0\\H_1:\beta_i \ne 0\end{cases}
  $$
  This means if the independent variable $X_i$ is significant to explain the variability of the quality rate ($Y$). So we have to standardize the estimator $\beta_i^{\text{hat}}$ in the scenario where the $H_0 =$ is true:
  $$
  \beta_i^{ \text{hat} } \in N(\beta_i, S_{\beta_i^{\text{hat}}}) \rightarrow t = \frac{ \beta_i^{ \text{hat} } - \beta_i}{S_{\beta_i^{\text{hat}}}} = \frac{ \beta_i^{ \text{hat} }}{S_{\beta_i^{\text{hat}}}}
  $$

  Then the third column is calculated as the division of the first and second one: $t=\frac{\text{Estimate}}{\text{Std. Error}}$. Finally, this value is used to compute the $p$-value to know if it's a significant hypothesis contrast (i.e. accept $H_1$) or not.
  $$
  p\text{-value} < [0.01, 0.05, 0.1]  \rightarrow \text{Accept }H_1 \rightarrow \text{This variable is significant at the time to explain the variability of } Y
  $$
  And as we can see, we only get significant values for the variables: ripeness_index, sugar_content_brix, firmness_kgf and length_cm.

* Residual standard error: It's the square root of the Mean of Sum of Squared Residuals (MSSR), that means the average error made by the model.
* Multiple R-squared ($R^2$): It's a indicator of predictive indicator of the model, since it represents the percentage of total variance that is explained for the model:
  $$
  R^2 = \frac{MSS(E)}{MSS(G)}:\quad E \rightarrow \text{Explained};G \rightarrow \text{Global}
  $$